In [47]:
from numpy.random import poisson
from numpy import log, inf


In [5]:
#  Input: x - a string or array of integers
# Output: a boolean array
def to_bool_array(x):
    return [bool(int(i)) for i in x]

#  Input: x - a boolean array
# Output: a string of bits
def to_string(x):
    return ''.join([str(int(i)) for i in x])

#  Input: n - the number of elements
# Output: a map of interleaved locations
def interleaver_map(N):
    pi_map = [None for i in range(N)]
    pi_map[0] = 0
    b = 221
    for i in range(1,N):
        pi_map[i] = (pi_map[i-1] + b)%N
        b = (b+420)%N
    return pi_map

#  Input: U - information bits + CRC32 + 00 
# Output: X - 1/2 rate convolutional code
#    bit n turns into bits (n^(n-2)) + (n^(n-1)^(n-2))
def outer_enc(U):
    X = []
    for i in range(len(U)):
        X.append(bool(int(U[i]) ^ int(U[i-1])))
        X.append(bool(int(U[i]) ^ int(U[i-1]) ^ int(U[i-2])))
    return X

#  Input: X - convolutional encoded data
# Output: A - interleaved convolutional code
#    remaps each bit in X to a different location
def interleave(X):
    pi = interleaver_map(len(X))
    return [X[i] for i in pi]

#  Input: A - interleaved convolutional code
# Output: C - PPM symbols in 16 bit channels
#    every 4 bits is accumulated and mapped to a PPM symbol
def inner_enc(A):
    C = []
    PPM = 0
    for i in range(len(A)):
        PPM = 2*PPM*(int(i%4 != 0)) + (PPM & 1 ^ int(A[i]))
        if i%4 == 3:
            C += [bool(int(j)) for j in '{:04b}'.format(PPM)]
            PPM = PPM & 1
    return C

def channel(C,Ks,Kb,M,P):
    Y = [None for i in range((M+P)*len(C)//4)]
    for i in range(len(C)//4):
        for j in range(M+P):
            Y[(M+P)*i + j] = poisson(Kb)
        Y[(M+P)*i + int(to_string(C[i*4:i*4+4]),2)] += poisson(Ks)
    return Y

def log_likelihood(Y,Ks,Kb):
    llr = []
    for i in range(len(Y)):
        llr.append(Y[i]*log(1+Ks/Kb))
    return llr

def send(C,Ks,Kb,M,P):
    return log_likelihood(channel(C,Ks,Kb,M,P),Ks,Kb)

In [2]:
U = "011000011111010111011000100100100101001011001011100100011011010111110010001100000011110001111100001100101000101010001010100111001011010110101110000101100110111101010111001001100011001011001111111001110101001001100010100011101010111100111010010011111111101110001101100010001010011001011111011011101100110111000111101100101001010101000011001110110111110010011111000110111011001011010000111000011100000111001010111100111101100111011111001011111010010011010110111111110001101000111000011101011111100011111010000110000110100110000101111101010111000010000000100011001010001101010001010001010011011011100011010001110000110001100000110111010110101110011101011001001011110010010011110100001111000100100100010111010000100111001111101110111100110100000001101000110110110011110110111100100000001110010110111111100101000100110010011110010011100100100000010010000101111011111100001110000111100111000100100010000110110111100010010011010100001111101110100100101010001111111011011101110110001111111001001101000001010100101110111011101000110110101111011110000011011010101000100001101010101101111100100110000101011000001000100010111101100000111000100011001001011001000100111100001111010011111110110001110101001000100100101011000000101110001011100101001000000101010011011011100000011011010010001111110100101010000000010111011110111010110011000000110101000111111101001100101111110011000001001000101011100011011111110011100111111101100111010111111111010000111001010001100000001111011111000011101110001100001011101100101101111101100011010100000000111101010011000011101001001010011110000110101110101100001100101100000001110110100000100011001000111011110110100111111111101011110100000011011010111111111011011010110111110010001000000100000100111000101000100001100101101110011010101011111001001010011100110100010000100011110111010011000101101101010110101000101001110111010001110010101111110001011011010011101110101000000100000000011011001010101011100001100100100000001011010100000011110110111100111100011000000110011110011101101111001110100010001011000110101001000001001010110110101011000001010001100111000000000100000101110101111111111011010101111101111100001001010010011100011000101110011100101100010011000110101010101111000100000110111100010000111111111011101101001110001010100111001100000101110101101100111101110011001000101100100101010010010110001101011010111111010111001011111011010111101001101001101011110001101010101111100100111111001100110000000111001011111101100110010110100100100001000010111011011000100111010001011011001110111111111101001110001111010000010110000010010000011001000110100111111000101111100110001010000010101011011111010110001101111011001110001001110110000011100000010011001001010110100000001100110001000001011001101111100001011001001101001010011001010101111110011010100111000000111100000011110011111010011001110010101001011111001000110011101000011010001100101100110110111111000111011111101101010101001010001000100011000111100011100101111110011000110111101010000111000101100010110101001011001100001100101101100010100100000101110100101011101110100111011110001001000001100111010101010010100100111011010100110110111001001110011010111110111101001101001010001101010110111100100111101001100110000011000110100000000110011000011101000010010001100000000110110000010111000000011111000110010000110010000111010111101001101001011011101110111001000010001011001110100111110000111101111010101101001100011010110100000000011100100011000110101010110110111101101110001100111001011100001101110001010010110110010110011000001011100000111101001011100111111100000100111110011010100100010101001001010000001000101001011011110001111010100111111001001101011111001110000010110001110111110100110111001100011001000101110001010001100001000010001000011001011110011111101101001110000110101011100111001110001100110010110111010110000110001000001100101000111011001011100010100101000011001001110001010111101010010110000111001011011001010010010111001110011011001101100110000011101110111011100011110011100111101001011111010101100001010001010010001111011011011100010001001110100001000011001011100100110100001001110011101111100111010000001101110110001001000011101101111100111100111111111010001010110101001001110100001101101100011100100010001110011001001011101100001110000000000001000010011111100011001101001101010110100110101000111011001111101000001100001101010010110111110110010110110011101000001111000111000000010110110001100110001010100000100111100100000010010101000111111111110100111000001101011110001011100100101110010101110001011111011000001111001000000010000110111101011100000110101111110010110101000000111110101110110010001000110111011000100001110111110000000011100111110011000010100001010000101000010011010001101100101111101110011011000010010001001110110110111111011011001100010010001100100011110010010111110001011011000011101001110011111111100110001000001100001101110111111011000110110010100100001111001001111111101000010100110011010010010011101010011111101100100000101110100100111101001010011100101010010111101111101100101011100001000101111011110010101111101000110001101011000111010101110010011110100111110110000010110101101000110101011010111101101000111111000001110010011100000111001000110101101110110100101001110011010110110111110100000110111111110011111011111110100100010101001110000011010011000000000000111111100101000010111111010100000110111010101101000111000010011100100110100111001010111111100101001110000110001111100110111111001101110100000110110010111100100001011000111111101110111110111011111100111110011111101100101101010100001110110000101101011001100011001110100000101100011011011010100000100000001011101100100100001000001111101111101101101111110011101100100011100001110100011011011110110111010010010100110011001011010100101101110100011011011111110001001100000001000000111111100110111010110101000101110111100000101110010000010001000111100000011001000101100000101011100100001111010010110011001011010000111010111101000100100011010001011101000010100101001001100100110100110101000011011000001011111111011000001110100111010111010111010001100100101111100100111011001101000110111001110010011001010111111111000001000110111011010101000100100100011110101011111011111100110000011101001100110001000010010001111010100011000101111000111000100101000110011111101101001001100101101111011101111000101101011100111001010000100101100000101000101101000001101110101011101010110000000100000110100100010011010001111001100001101101000011011101100001010011110001101101110001011000011100001011111110111011010111011101110000010110100110100101000011010010011001100011100000111111101011101001010100001100001111101001111100010101010110100001111110110100011110000001011100010111011100101111101011100011110011100000011100100100001110010010110100011100101011011101101110000010001010111000111100001110110110011111110001101100101011100000000110001010011111001101001101001011110110100110001111000100110111000010100110001011001011010111010100011110110100001001110000010000110101000101100101010101100110011011001110100001101011100101110001110100001111100110011000110011010111010010011100101110101010111010110101100111111110110100000010011001111100111110010111110111001111000000001011001110110110111111001000011100001011011000110110011010110111010001001110100110110001101000111011011110010101010001000100010001100100001111010110010100011101011100101011110010100100001011110100010001110001111011100011101110001011111111001100001000001010010010001000100110010011100000010111000011010010010001010101111101101001100100011110001110111110100100011000100000100001000011101010101001110010010011010011111000100101011000001011111100"
X = "001110101100001110010101100001001001100010101100110111110111110111110100011111010010101111010010011011110111001110100010100001001001010110111101110011101011000000001110010110110011100101011011000011101011110100011100110100010001110011010001000111111001101111010010100001001010000100100110110000110100101011111010001001011000010001001001101111011111101011001110101111010010101111100101010101101111100110000100011111011111101011001101000111001110011000010001001001011011111001100001111101111110010101010101011000100110110011101000101011001101110011010001111110101111010010010101100010100010011000101011111010001001101100111001011000101011110100011111010001000100011100001110101111100110001010001001010110111101111110010101101100111010001001100010101111010010100001110000111001101100001110011011000000111001101111010001001001011011111001011000101011111001100010010101101111010010010101100001111101111110100001001010001001010101010110110011101000011100111001101100001110011000010010010101011011001110010101100001110000111010110000111010000111111010110000110100100101011000010001001001101100001101110000000000110111001110101111010001110011101000010001110011010001110011010001111110100010100010011011001110100001110011100110110000111010110011101011000000111010001001100001001010000100100110111110011000010010101111011111010010010110111101111101111110010110000111000011100101101100110111110111110111001101001001100001110000110111111001101111100101011000100110001001011011111010000111000000000011101000011100111010001010001010111110010110001010001001011011110111000000000011100110111101001010001001010101011011110100011100110111111010111101111110010110111101111110011011110111110111000000001101111101110000110100100101100010010101011011000011100110110000111001011011111001101100110111110111001101110000111010001010001001011011001101111101111110100001000111000011100101011000100110000111110111110100010001110011100101010101100010100010011000100110001010110011100101010101101111011111101000011100000011010001000111110100100110001001100010011000011100111010001010000100100101100010010110110000001110100010100001000100011100110111000011101000010001000100101000100101011011110111111010110000110100010010101100000011011100110111001101001001011000101011000000111001101100110111001110101111011111010010101111011100110111111001011011000011100101100001111110010101010110001010110011100110000100011111011100110111110111110100010010101100000000110100100110110011010010011011110100011111011100000000110100010001111110100010100010011011000000001110100010100001111101110011100101010110000111110100010001110000000000001101001001100010010110001001100001001010111110101100000000111010000100011100111001010101011000011111101011110100100101010110111110101100000011011111011100110100010010011011001110100010010101010110111110011011111001010101011000101011111001100001001001010101010101100001110000111001101111010001110011101011000000000011100101100010010101101100001110011000100110110011101011000011010010011000101011110100101000100101011000101011001110100001000111000000000000111001011000010001111110101100001110011000011111011111010001111110010110110000111010000100100110000100101011000011101011110100101011000000000011100110001010000111000000110111001110101111011100111001100010010110001010000111111001010101010101011000010010010110000111000000001110100010100001001001010101010101100010100010100001001010001001010110111101110011011100000000110111000000110111111001101100110100011100110111000011101011110100101000100110111110100001000100010010010101101111011111010001111110011011111010000111001101110000110111001110010110001001100001111110101100110100101000101000010001001010000100011100110100011111100110001001100001110011100110111101000100100101010110110011010010100010100001111110011000100110000100011100000000110111000000000000001110100010101111010001000100010010011011000011101011110111110111000000000011010010100001000111000000001110010110001010001001011011111001011011001110101100000000111010111110010110111110011000101000100101101111100110000111001101110011010010101100111010000100011111011100000011011111010001001010001010000100010010101100000011010001110011101011111001101100000000000000110111000000110100100110000100100101010101010101100010100001000100100101011000100101011011000011011111010001111101111110011011001110101100110100100110111110011011110100101011001101111110101100111010000100010001000100100101101100110111000000111010001001011011001101110000111001010101010101100010011000101000011111100110110011010001000111111001101111101011000000110100100110000100101000101011111001011000100110111110101111011100110100101011110111110100010001111101111101001010110011101000010010100001001001010101100001001001101111010010010101100010100001001001011000011111101000011111101000010010010110110011101000010001000100100101011011110111111001010101101111101011111010110000000000111001101111010010010101011000101011111010111101001010000111110111110111000011011100001101001001100010100010101100110111111001100001110011010010100010101111100110001001010101010101011000011111100110110011100101100001110000001101001010110000001101111101110000001110101111011100111010000111111001010101101100110100100101011011111010110011010001110000001101000100010010100010010101100001001010110011101000100101100010101111100110110011011111100110001010110000001110011011000000001101111110101111011111010001001010000111000000000011101011111010110011011100000011010010101111101000100101011011000011010010101111011111101000011111010001111110101111010001000100100101010110111110100001000111111001101100000000111001011011000000001110010110111110010101100001111110101111100110111101000100011111010010010101101111011100111010111110011000011100001110100001110011101011110100101011111010001010001001010101101100111001100010010101011000101000010001000100011111010001110011011100110111001110101100111001011011001110011011110100100101010110111110101100111010001001011000010001110000111001101100110100101011001101001010000100011111010010101111101011000011101011110100101000101011001101000111110111000000110100100110000111110100010010011000100110000111111001100010010110110011011111011100000011101011111001100001000100010001111101000111110111111001100010100001000111111010001010001001101111011111100110111110100001001001010110001001011000011111101000011111010001110011101000010001001010001001011011110111111001011000011111101011111010110000001110101100111010000111000000000000111010111110101100001110011000011100001101111101110011101011000000000000111010001010110000001101001001101100000000001110010101101100111010111101110000111010111101110000111001100001001001011000011111101000011111010010100010011000100110001001101111011100001101110011010010101111100110000111111001010110110000111001011000100101100001000100101000011111101011001110100001001010000111000000000000001110011011110111001110101100111010000100010001001010001010001001011000101000100110110011101011111001101111010010011011000011101000100110110011010001111101001010001010111101001010111110101100000011010010011011000000111001011000011111010010011011111001010101011011000000110111111001010110111110100001000111110111001101000100011111011111010001110000000011011100110100011111010010100010010110110011100101100001000111111001010101101111011111101000010010010101101111100110110000001101001010110011100110001001010110000111111010001001101111101011001110101111011100110100100110110011010001110011101011000011011100001101110011011100001110101111010010010110111110010101011000101000011111100110110000111010000100010010011011111001101111100110110011101011111010111101001010001001100001001010110000111010110011011100000011101011110100011100111001100010101111010010011011001101000111110100011100001110101111011111100110110011010001001001011000010001111101001010110000111001101111010010100010101111010001111101111101001001101111100110111110100010101111101000101011111010110000001110011000100110001001100010011011001110010110111110011011111001011000011111010010010101100001000100101011000011010001110011010001111101110011100101100010100010100010011011001101110011011111100110000111000011011100001110101111010010011011110111111010000111000011011111100110111110011000100101011011111001100001110000000011101000100110001010110011011111011100001110011000101000100101011011111001011011111001010101010101100001110011010001001010000100011111011111100110000111000011101000101000101011001110011011110111001101110011100110111110101111011111010010011000101011000011100110110000000000000000000011011100001101111110010101011011001110101111101000011111101000010001001010000111111010000100011100111001100010101111100101011000011100000011101011000011101000010001111101001010001001010110001010111101001010001010111110011000011100000011100101101100111001101100000000001101001010001010110011101011111010110011010001000111000000110111111001011011110111000000001101111101000100011100111001010101010101010110000111111001101100000011101000010010010110110011010010011011110111110100100110111101000100100110110011010010010101100010101100000011100101101111011100000000001101110000111010001001011000010010011011000000111010000100100101010110111101001010000100011100000000111001010110000100100110001010111101110011011100111010001001100010101100110111000011100110001001010110110000000000001110011011111001010110111110101100001101000111000011010001110000110100011100001101111110100001110011101000101011110100100101011000100110111110100010101100001101111101110011011111100110001010001010001001010101100010100010101111101011001101111101110011101011110111001110010110111101111101001001010110110011010010100010101100001110011000011111100110111110010101010101011011111010110011011100000011101011000011101000100110001001010101100010101100111010001010111101000111110111000011100101101111011111100101010101011000011100001101000111111010111110100001111101111101111110011000010001111110010101011000101011110111000000110100100110000111110111111001011000011111010001111110011011110100010001111101001001011000100101011000101011110100010010011011000011011100110100100101100010010110111101000100100101011000011100111010110011101000010010101100111001100001000100100110111101111110010110000111111001010110001010110000001101001010000100101000011100111010000100010010100001001001011000101000011100111001010101101100000011100110111101111110011011000000111001101111011100111010000100101000100110001010000111110100011111100110111110100001001010001010001001010110000111000000111010001001010101010110111110010101100010010101010110000111110111001101000100011111100110110000001110100001111110101100000000000000000000111001010101011011110100011100001101001001010101100001000111000000111010001001100001000100101000011100111001101100001101111110011011110111111010000111111001101111010001001001010101011011110100011111100110110000111010110011100101011011111010001001010101101111101000100110000111000000111010001010111101001001011011110111000011010010101100111001010101011000100110001001010110001001100010010101011011111001010110111110010101011000101011110100101000010001000111000011100110001010110000110100101000010010101111101011001110101111100110000111000000110100101011001110100010100010100001000111000000110111000000000011010010011000101011110111110111000011011100000011100101011000100101011000101000101000100101010110111110011000101011110111001110011011000011100110000111001110100010100010010110001010001001100001111101111101000111111010111110101111010010100001000111110100101000100110000111001110100010100010010101010110110011011111101011000000000011011100000000111001010101011011111010001001100001001010000100011100110100100110001001011011000000110100100110111101110000001101110011011100111001011011000000001110101111011100110100101011000000110100010010011011110111000011100101100001111101001010111110101111010010100001110000111001100001001001011000011100110111110111001110100001110011010010011000011100001101000111110100011111011111101011110111111010000111111010000100011100001110100010101100000011010010010101010101100010101100000011100110000111111001100001001001100001001001100001110011101011110111110100100101011011110111111001100010101111101000011100111010001001101111100110111101111110101111010001001001010101010101101100000011011100111010001001100010100001000100011100110111110111110111001110010110000100010010010101100010010101011011111010110000001110011000011111101011111010110011011100001101111101110011100101100001000111001110101100110100100101101100111001101100110111110100011100111010111110010101011000101000011111011111101011110100101000100101100010011000100101101100110100101000010010011011111001101111010001110000110111110100101011000000110100011100110100101000011100000011101000100110000100010010011000010001001010110000000000110111000000111010000111110111001101111110100001110011100101101111101011000011101000101000011100001110100010011000101011000011010001111110010110110011101000101000100110110011010010101100001110011011000011010010010101010110001001100010100001001001100010011000100110110000001101001010000111111010000111110100011100001110100001111101111110101111101011001110011011000000111001010101011000010010011000011111010001000111000011101011000011100101011000011111100101011011001101000100010001001010000111000011100101010110001010000111001110010110110000000011010010011011001101001001100010011011110100100101011000010010011011001110010110111110011011000000001110011011110111110111000011100110111101111101001010000111001110011011110100010010100010011000101000100110110000001101110011010001001001101100111001011011000011100110001010001010111110010101010110110011101000101011110100010010011011000000000000111010110011010001111110010101101111101000011111101000011111010010010110001010000111111010110011100101101100110111111010001001101100001101000111111010110011010010101111010010100001001001100001000111001110010110001010000111000011011111100110110000001101110000111010000100011100110100101011110100010001000100101011111010111110100010101111100110000111000011101000010010011011110100100110110011100110000111000011100101011011111010111110101100111010111110100001001001100001111101111110011011110100100110000100010001001001100001001010000100101011111001010101010110001010000111000000001101111110101111100101011011111001010110111101001001010110001001101111100101101100000000000011010010101111100110001010001010001001010101101111011100001110011011000011010010100010101100111010001010111110100001001010001001100001110011011111100110000111111010001010110011101000011100111001100010100010010110111101000100010001110011011100110111001101110011101011110111000011100101100001001010111101000111001110011000010010011011110100010010010110111101000111110111000011010010010110000111001101110011100110110011100101100010011011001110011000100110110011010010010101010101101111101011000011011100000011010001111101111101110011011100110111111010111101111110011011000000001101001001101100001110100001111101111101110011010001000100100101011000101000011111101011110111001110010110110011100110001001010110000111110111001110101100110111000000110111000011011100001110011000010001000100011111100110111101111101111110100001111110010101101100110111110100010010101100000011010010010101011011"
A = "000011000001001100111010110010100100100001100110111000111000000010110010001100001110011101110101110001101001011110101001001010100100011001110100001101010000010011111111011110111011010010010010100001010010111011111101010111111011010100100000101001101101100101011011010111010011011010010110100011010111001000100110111010010100001111001000111110000101000111100110000101101111000010111110111001111010101110111001111010111000011111011100111001111010100111111000011110010011010011011100100100111111001111000011100000011111010000100110110000001111110001100001000001011111111010011001111010000111101000001010010111000110111111001000000111101001001010010100001001101110101000010100101110001100110000000101010001010010010100110110000001010000010101111100000110011110010011010101111000101111101000001111111110100000010010001010000010010101011110101001010001111001001101101111111100111010011111110011010011000010001010011111001000101010010111101100111111111001000110010100011001101111010101001100001011101100111011011010100100011100011101111110100001101111011011000010011111000010111111011000111110100101111011101110000110100101101111011000111010100010111000101011011110101010000001011111100001000110011101000110010101101111111100111010100010011001101111001000100011110010011110110010111000110111011100111100001111111011000000100010101100010111111011000110111000100101110101010110111000110000111111011000010100100100011000001001001110000010011001011011101100101100111000000010011000100001110011011000111110001111100100111000010001010000110111101000110111011111001110010111100001010110111011101000101010001001111111100010101000001000101001101010111011001100000011010001000001001001101101000011110010101101101011101010111100101100000000000101101000100101000011100000110000011010110010011011010110100001111000100101110011100110101110011011010001111010000010100011011111110010110111111011001001111111110110001010010010101011110011000011100000000001111100010111000011100000000111000110011011100010000100010010011101010000001110000111001000111011000001001111101011101011110010111001100110000110010001000111110111010110010110111000101100111100100011000101100100100110111111101100011000101011110101100000010001100101110101011001111110010010011010011011010110011110101101010111010101010011101010100000010000011101111001000001110101000101111010101010001111111100001010100011110000000110101011111010001110101100001010001000010110111111111111010001010000101011100101111110110101110100000001010010100110111110001011100011100011111100111110110110100001101110111100011010011110001010100000000000111011001000111111010011111001011001101000101111110001110001001001010101101111111011101000111110101010100100110101010011110011101001010101010100100011100010100100100011110111101111011110100010110010010101111011101010001011101101101011110011011110000110000111101100001011000010110001000111100111111011010111100001011000001111100100111101110001100011101011101011011010100100010111100010111111000011101110000101111000001101011001001100000000001101100010111010111010111110010100101010000111010101010110110010111110100001111110111110110011000010011111001101011101111001010011000100011011110011011110111011110101001001101100011001010011001000100011110101000101111100101111010110001010011000011101000010110011001010011010100110110000000100110011001011100011110010000110110111100100101110101101101011110011101111001001101110110010001001001011010100000010001000010010010111000010010001111111100110110001100111011100000011010000010101110000110100001011011001001101011111111100000101001000111001110000111111100010011111001001101011000011100110100011111100111111010010110100010010001111001010111001101111000111110011111100010000011111100000011101010110110000000001110101010001110110001010101101000010001000010011001110100110100101010000001100001101000010010101100000001001101001010100100011100010011110010101001111110100111011010101011101000100010110101010110100001001111101111101001110011100001101111101001000001000001101001001111101110010111000101010100011011110001101100011000000010100101010101101000011100001010110010101101000101011100111011111001101001110111010111111100111101100101111001001001000011101010100010110100001011110111001001010110010011001011001011011110000011100101011000100110001101011110010011101111010110100100000111101110101101000000110111111111111101011001101011110111111000011001101011101001101001100100000100100101111001101010101011000100110100101010111101100110110101001111110110011101000110010010100001110110100001000101110111001011010010100010011111001101111010011001001000100010010100111010001110011100100111001111100101000001000011101011001011111010010000011110101100001000011111010011110101000011110010101011000111001110011111011111011011100000011001101011011110011011101011011010101011101001111000101010101010000011100111101101001001010110001001010110001010001100010101101010100010011001001101010001011010101001000100111000000100000111101010100000111011111010000001111001110110001011010101101001010011011000010100101110101111011011100100111001001110011111000001010011000101111100100011101010011110101010011011111001011100010001000010101011011010100111010001110101010010001110110001001001011001011100110110001101011100100010010111000100011011010110101000111000111100011000100111100101001011010110101010111001110000101000110110001011100100001110111110010010110110000001000010101011010001111001001100110000111101101101000110001101010001011011101011101111011101100110100110111010001100111110100100101111100111101001010000101010001011001011111000000100100111001000111010111100110010010110100110000110011110000011011001110011001001001001101000110111101001110010011001011110110011010110010001001011011010000111110100011011011101101001000001001000011110011000111001001110010010100011010011000111101100010011010110011001000100000010100111001101101111000110101010100000001111101110000010011001011100111100101000010110011101000010110100101111100100010000011100010111010111010000010001001010000000110111011010001101100101001101000011010110011000011000100000001101111010110111001100001101001111001001000100010000110010010101010000101000000010111011001101111101001111101000010000110001011001000101111000000011100110100110010110011111110011101001001010110110001011000001101001010110011101000000110110010010100001011010011000001001100110011111010001010011001011101101100000111110111101000001111001001001011011010101010111111010111001100101010001000000000010100100011011010011101111100011100010011111111100010100011001001010010111000101101101100001001100101000011011010000100111001110111000010001100010010011011110100101111011100111100101000001011001010100011100001101010011110001101111011111110001100110000111100100111000110100010000110000101011000111111000001111101111111010010000110010101101111011010010011111010000001011110101001101000010000110001000101011111011110110000101101110010010011100110110001111110010000001010111001111001110110010000111011010001110111010001011011100100001000010011100101110000101111101110100000000111000101101001010100110100010101011100100000111110010000011011101101101100000001010110001111100111100111111011010101011110010011011011111100001100101000101000100101001110100000000110001000100010101110111101110101001110001001110010011110110001110011000100100100000000011001110010111110001110011001000011101111010110111011101111111111111100101000100101111100000101111011000100011100010101011011101010111110001101011001111001100100100011001010110011010101111001110001010000010111011101000000000000101010101001000101010101110101111100100101111110100101001001111001001100111011111010110001111101001100111101001000111100110010011111101001111101011110110010001111000011000110000101001101101110110000101100101111000111111011110011000001111001101101001101011010011111110101111011001101100100001111100010010111010110101000010101000001000101000001101101011010011000000110111000100010011010101101010011100101011110100000101000010111010000010101000010000010001111011101111110000100001001111101001001000000101010010001001000111010101011010010001110011001101010011111001010011111001101101110000001010100010111000000110010100000000110111101000010001010110101001000101010001111110111000001010001011001110011110011011011000001100001111011100011001111011000000110011100101110010010011100101111101100100001111110001100001111010011010110101101011100110111100101101001010010001111000100101101000010111001111101101000001000001001111111001000010010110111011010111011111101111001001101001000111000000101001010111010011110010010110010110100110010100100110010010000100111010001010011101010110011010011001101111110010100001110001010010011100010001111010100110001100100101111000011011000111011100100111101101001100010001110101001100001100010011110100010100110011101001101110110101100001011001001111100010101011011001101110010000001100000001100001111100110010011010010101000101101010010000000100100100011011110000001100101100101100001000101100001011111011010100100011001110110001110101111011100011100101110101111111111001010010000011001011111000000001110001110100000010011100111000000010101100110100011010001001101110100110111111011001100101011100011011001011100111100000101100101000110000010111011001001111101110001000000011000010101111010001101101000111000001100000101000110110011111100011001001011110010100100001100011001101100111110010001111100101110011111100101010110101111101000111111011111000010110100010000100001110010001000101101010011111110000110011111001101111101000011111001001001011111110000100111001101001110100101010100101010011010101010101001001111010111100000000100010001010000001100110110011101001011110100011110100110101011001001110111110001001111011011110001101101110111101100111000110011100110011000011000000110100000100000100100101100111100110110001001000111110010001010110010011110110000101101111011001001100101110000011011001100011011111110001011111111100101101110001011011100100001011001001011100011010100011000101111101101000011110010011101101010000010001100101100011010100011101000011010101010111000101001001101100101101110011001011001111111010011110001010001101101110010010111000010010001100000001111011110110110011011010101001111001010000101110111000111101001110101000001010100100001011110001010110101100101100011010111010100001110100010010001100010100011111111111101100111011001111100011101001110110000101100110100100000010100011000101101000101001111001010000111100000101101000100010100100101001110011111001110011100001100000101000010111010000010000011111000011011010011110111111001100000111100001011101100111100110000101010110101110111110101001100101100110001110110110011001101101101010111001111000101011101010101101001110001111001100000111010001100101000111110011110010010110001010000111000011101100111000001101011110100001010011000010101011111101100110001000110101111110011010110000100111111110010010000101101011111110101000001011010110001100000000010101110011011011101111111000001110001111001100000000000110001100010110110011011100110001101011111100001011010100010011100000001001011001001100101011000101111111101110101110110101100010110111100110111111001110011101111001011010011011011101111100000010010111010011111110011101001010111000100111100110110001110010110110101101010110000110111110011000100001100100100101111001111110101010111000111110011000001111000001010111111100001111111100011011000101000010001110011010010000110001001011101001000111110011100000000100100010101000010010111100100001000111011100111100000000010000001011111100111010100111011001111100000111101110010111011001011010110001000001100010010000000111110010001111111111110010001111111000101010100000011011101111001000100100101001010111011001011111111011011000111000100110000000000010011110110001101000101011111011001000100101101110101110110011010111000101110111010100001010111100001101000111010001010001101101101101001100001101010100000001010111100000110110100100010110101000100100001111100111100101001101001010110001110111011111101000110110100101100101110111100000101011111101111111101011001001110101111000101010010010001100100001110000101100010011011011000101100110100011111001000010000000101011000011010110010010101100001001001110001001100000101011101011011111011110100111111101000100001110000010110111011110111100011001111000110000100001010100101111111001011001111111111100011000110001101100111011011001100100110110001110010101000000000010101011111000010111101001011110100100111110110011011001110000110101111110011000011010011011111010001001100110111100011101100001101100110110110111011010101010111011011110001001000110111010111110100110101001101011101010111101100011010100000001111111110111100011011111010111011100100110011010011001011100110001010011100111001100011001001101100011011011110101101011000010001111100111111111010100000010100011011001000110100111010111100000101111000111000010101010100001111011111101011110111110101010111111000100101111111000010001010111011001000110110011110110100011111111110001010001001011101111110001001010000000110011010111011000100100010001001100101011101111011111010011111001101101011100110111111010011111101010000010010101010100000001110100000100011100110010100110000100000100100000100011011011010011100111110111101100101000110000010010011011001010100101100110001001000101111111011001010001000101011010101111100100100010101010000111100001000001000110001110010100100000101001001110111111001001000100110000010001101100010001000010001101010001010011001000000011110100011011000111110101001101000101011010101111001101111010011100001111010010000100100110001101111101110101001111101001111100110001110010011011101100101001011010100011000000001001111110100011000110101100111100010000011001110001110111111001100010010111011111001110101011010100110001000010000100010011001100011000010110011111111011001001011000000000010011111011011101011010000001010001000001100110011100110101101000110100101001110110110010011001000110111101110001110011010100011100000111101101001110000010001001010000100111000010100011010111110101101111001101110010111100011001101101101001100001100001111001000111101101011001111010000010101101011001101001110100100110100111100000101011101001010000000010101010001101101000101000011000110111100111010101101011110110110010000100001011000100011100010100011100011101001110110101000110000001010110010100100111010110010001010110001101101101011100010011110111110011010110111010110110000010100101110000101011111101001001110100111111000001010110011011001010100100010111100011000011001000000111100001011101001111111000111100111110100110111001001010100101011001010010101001011001001110100111010111100111000100010001100110110000100110001011100011100111001101111011011011010100100110000101011000100010010100000011000010011010001010101010010001010101000101110110000011010101111001101110101111011101101001101010011100001100011101010010101011011100100010100011011000110111000001001000010010011110011000001011100001100010011001011101000111110011000001001110011001000111010100000110000101000010110100101110010111111010000101101111011000101010110010110000111100110110101001011111111110100100011111110101000110110001010000010011101111000000101111000100000100101001010011100010111111010011110110100001000110001000000110110"
C = "000010000001110111010011011100111000111110111011010000101111111100100011110111110100010110100110100001001110010100110001110011000111101110100111110110011111100010101010010100101101100011100011000001100011010010101001100101010010011000111111001110110110111001101101100101100010010011100100111101100101110000111011010011100111110101110000101011111001111010111011111001001010000011010100101110101100110100101110101100101111101010010111010001010011000101010000010100011101100010010111000111010101110101111101000000010101100000111011011111110101011110111110000001101010101100010001010011111010110000001100011010000100101010001111111010110001110011100111110001001011001111100111001011110111011111111001100001100011100111011011111110011111100110101000000100010100011101100110101111001010110000001010101011000000011100001100000011100110010100110001100001010001110110110101010111010011101010100010011101111100001100010101110000110011100101001000101010101110000100011000010001001010011001110111110010110111010010010011000111101000010110101011000001001010010010000011101010000011010101101111010100111001010010110100000100111001001010010000101100111100101111001101101011001100000001101010111110000100010110000100011001001010101000101100111100010001001010001111000010100011101011011100101111011010010111010111110101010010000000111100110111100101010010000100101111000110100110011011010000100000101010010000011000111000010000001110001011111100010001101101001000110111010000000011101111000001011101101111010100001010111000101111100001100000100101001111011010010101110100011010111110011011010010110000110011110001010101000011001111110000110001001100101101110111111101100001111110001110110110000010100011001001001101001100101000110111111111111001001111000110000010111111011111101100100011101101100100111110101111000110100010111011001011101101100001010011111100111101101010100011011010101101110001010101011011110011100011001101011101111101000000000001010111100101111101000000000101111011101101000011111000011100010110011111110100000101110000101101111110001010110010110010100011010001000100000100011110000101011010011011100100101111001000101000111101111001000111000100101010110111101111001101011001000000011110111001011001101110101011100011101100010010011011101011001001100101100110011101001100111111100000010110101110000001011001111001010011001100001010101000001100111101011111111011001101010011110100110111110011110000011011010101010101100001100000110010111001010100100110100111111110011100111011010100001101000010111101010111010100100100111110110100101000010011101011110011000000000000101101110000101010011101010001101110110000110101011110100001110001100110110101010010110000101011001100111000100110011101011101001110011001100111000010111100111000111101011010110101101011000011011100011001010010110011110010110110110010100010010100000100000101001000001101111100100001111010111010101101100101000001101111110101000111010110100001000010110010110010010011000111100101000011010101111101001011111001010000001001101110001000000000001001000011010011010011010100011000110011111010011001100100100011010100111110101011010100100010000011101010001001101001010001100010000111101101011101101011010010100110001110110111101110011101110000111101011001111001010111001010011011110011101111101001111100100010001100010011000100100000000111011101110010111101011100000100100101000111001011001001001101011101001010001110110100100011110001110010011000000011110000011100011010000011100001010101000100100001000101101000000010011111100110100000100111110010010001110110010101010111111001110000101110100000101010111100010101110001001101111101000100111101010111010101100011011000011100001010001100101110110101111010100010101000011111101010111111101001100100100000000001011001100001011011110011001001111100001111100010001011000100111001100000001000001001111100011001000000001110110001100111000010111100010100011001110101011000101101100110010110000111100100110011011000001110101001010110001011101000001001010110001111110000001001110001010110100011010000110011000010010100001001000010000000011000110011001001111101000001100100011001001111001101000101101010001001110100101100101010111010110111001010001110001111101001100111100100111110010100101110001100100011101110010001101101011111101000110010000111011110110010100011101001010011011000111111010110100110110000000100101010101010110010001001101011010101111101110110010110001001110111000000111000110101110110011001101111000100111001100101001000100100110001010100100010110000100011100111110100100111110000110100101110010011100111100010101110110101100010001110000111100011000101100001011101000111010001010111001111110000010110010001101010011100000010100110111110000010101100010100110000010100011001101111010001011101010010101101101000000010001001101101011101101001101101100110010110001010000110011001100000010111010110110001110011011110001100100001100001000011001001100111100010001110110011110010011001110000111010000000111111010110011000000101101010011111110101110100100001101100110110001100010010000011000110100110101101101000111010001110100010101111110011101111001010111000010110011101011001100010010101110010111100001111100110010010011000101100001011001100011110100100001110001101110010111011011110110010111000011100101111000010010011011001111010000101000010000111010111001110010011011001100101110100000110000100100001101000111110100101011100011011011111110000011001101100001010001110111011111010110110110000100001001100001101101001101001010010110111011000100101100001000101011000111001010111010110001100000110011110010001101010000000111000101110000101100101000100011100100111011111011101011111101101110100010001110001110110000100101001110100011101110010100100010011011100001110010010011111010100111101101101001001110000001110000010100010000101110001011100011000010011101111010110111100010011011101110000111111100111010001001001010000100110011000000001010110100000011101110010111010111001111100100010110000011011000110101000111100000010111100101100101100000011110001100000000100101101100001001000110001001111101100100010000010000111111110110101100100101110111110110001010001110000111100000100011100110011111001111111100101101110110101001110101001111100000100001101110000110101111111101000100111011100100010101011101001110001100100100001101111110110001100100010110000000100100011100111110010011101111110001000100010101100001100010001101001001000000101011010110000001010001110001101101100110011010101100101110111001100001111111111100111000010010011101001010111101000011101010101000011000010001110011100101111001001001000001110111001111101101100000111010001011010000011110111100011101101011000110101101000101000110000001101110011000010111110110011101011110110101101010100001000100000101000111010000100111100000100000110010000101010000001010110101010011100000100011001001010010011100010101100000001101011001110110000011111011110000110010101101011011111001001011100011101000100100001010100011111110011010001010001011011100000101101100001011010011110010010111000001111100010111001011111001010110100111111111010000110110001100111011000011001101000111111010100011111101101001001001000000001100100001010111010111010101101100110010100011101101101010111110111001111001111000110001011000000000100001111000011001011010110100110001011110001011100010100100001011101111000111000000000010001011100101011110100010001111101001010011011010010110101010101010111001111000110101000000110101101111000010111100110010010110011010100001001101110101110111000111101110011011101100110101110100001100000011010010110000000000000110011001110000110011001011001010111000110101011000110001110101110001000101101010011011110101001110111010110001111010111011100010101001110101001101011011100001010000010000100000110001001001011011111001000110101111010101101011101111110101110110110001001101100010101011001010010001001000111110101000011100101100100110000011001111110000110000001001001101100010000000100101111000011101100110110011101000110010100111111001111100101100000011001111100000011110101101001010100000111110001010110001110000000110011100001110000101100110010011100001011101110110011101010001100010101110110110100000001100111100101111111011100111111111011010110000011110011011001110000110011110101011010000001100001101110100010100010010010000001000001010010111101110101101111111011101000110100011100010111001010110111000001010100001000001010011101100100110110010111011010111001001110011100001010000111001001111100101110101001001111110000001110101010001111100011011010010011010010101001010001110110001111010000000110001100101100010100011100100011011000100011000111011100011111000101100001100010110011011101100010001001010100011000001011110011100010111100001010011000100001000111001010000010010000101101000111010110110001000011110100110001000001000011101011000011000100010110001001011011001000001101110001010111100110010010001001011100000001000000001000001010111011100010011100110000110110011100000000111000111101101011111110111001000110111110000110111110010101101100111000010001011011110100110101101000010111001011001010101010001100011111101110010101111111110100001011000000011101000101111111100110111011000010011110001001011000100101010010001000110010111101101110010111010111111001000110000100000011010010001110101001011110000000010000011001010011110110110000101111110111111001111011011101010111101110001101011100111000001000010001001000101011100001010111001011101010111001100100110101001111010101101010000011011000011111000001011100001111001001100010101011111011101010001001010110000010101110001110010101011111000101110110001011000110011000110011101100110011001110001010011010111111111000011110011111110111011011101001110010100111101011000100110010001110100101011110001010010010100001001001011010110111010000100010111011101111101111111011000000111111000111001000101000100100001110000101011100001100100011101011011111001001010010001110111001011111101101110111101101010100001101010101000110110100001101101000111110010001110010111101100111101111001010110110000010100011101001001100000011110111001000010011000010110000010011001100101111001110001001000110110100010001101110101010011101011110011110110110100011100101111100011110111111110101101011011011101101100110001010001100000110100101111010110001011001111110011000111110010100001100100110111001000010011010011000001011000011100001000011000010101010101001000101101110101000010110001011011111001000100111000000011000010000110110000110001010001100000101000000110110000111100111000110001011101010001011101000001000000110000011010011111100000010101111101101100010100101010001000000101000001101001000101000100000110011011001011010100110001000110111011110100100100010001001001001100101110101111001101001100110110001011110101110111111010011110111001111010100010100011100100001100000101111101001000101111110110010100111110011101111100110010101001000100001111011001010100010011011111000101010100011100000110110010101011001111110010011011110111111111100110100010010010110101010000001011110101110111111111111011110111100100100010010111011110110010101000001101100111100010111111110001101110001000110010000110101010110100110100100110111100100101000100101010001011101001010001101100010010010110101000000011100101100010101011101001110011010000111010111011011110100011011011001001100100000100101011101111000001000111000110101110101011001100101111010100010000001010000001100101010111110101010111101101111001111100001011101100011111011110001101001110000101011101000000000111000011001111100011010111000001111010010111010111111111100000001101010111010011000101101110101000000101001011100101101110010011011110000001000011100000000101011100001010101010100011110101010000110011000000010010110101110000111000110001100101101110010101010010010000101111000100000000000011101011011110110000110010101101110000111001001011001011011101100101111001011010011000001100101000001001111010011110011110110110110110001000001001100111111110011010111111011011000111100100110000111000001010111010111001110110001100100001011010010101001111011011000110111001011010111111001101010110101010110010001110100110101111001100011100001000111110100000110111100010010010000110111011000010101110000011111111001101111101100100011100110111110001110100001110111111001101001101101010010100111010101001111000001011111100100101101011010111101110101111011111000001100111001010101110010001010101010111101111011110110111010010010001000111011011110100011001111111111100110010101111100101001110010100111000101011011101101110100000100110101011101111101100010010101100001110111011010111101001000001001000100100100101101100110011010010010100001110000100101100101011000100110001001101001100101001000010011000000001010101011010111101101010011010010111000100010011101110010111011110011101000101110111101110001001000010010010100110110010000011110101000101010101100111111100111101101110000100111010011010111111001010000101111100110011000001010010101001101011010100110011010101111000110101010000011110011010010001111011011101011011000010101010100001100001110010110101011110001100000000100010011010010000111000011110001000110010110101101010011101010001001001101000100101010011101010110011111100011001100111111110100111111000010111011100111011111000000111000000111101101101100010111010100101001000110000100000011100010010001100111001000100001110000110101010010001100001111001101100110101000111000011001100000101000001111110000100001011100111000000110001110100101010001110000111011111100001001000011110000011110110011110011101110000000010100111101101111010100110001001111001101100110101110110101100010111110101100011111000111011110110101001011001110101001110101000100001011100010010110111001110010011000010000000001110101011000010000100110111010111100000010001011110100101010001000011100101101010001011001101100111011110000011111000011101110111101111100100010101010010001110010000000000011101010010010110010011111110011110000001000100010111011001001111011000110001011011011100010001111011010110100001011101100111101000000101001001110100000011110001100000111010000011000010011010100110110101110110100011010111101110110110110001000001000001010001111010110110010001010011111100110110010001001110100111000100111010111111001101001110011111111100110011110110110000110000010000100101000101100110110010100100100011111000001101111000010111100111101000010110001011011001111011111110011011100111000101100100011110011011110110110110010111100010100101011101100100101100100100000011000110100000110010101001110001011000101010000001100100010010001100111000011010111101111101110000000101000001101001110101010000101000101011000100101110001100111001101110011100110001101110001011000101100101000101111000011110111011011111000100001101000010111010001001010010010010011000111011111001101111000011100111111101111100010011110011001100011110011001111001011011111101100110101110110100110101101001001110110011101000001000010110011100110010010111000011000010010000100101111110001111100011101011101111110010111110111100010001101001111010100010000001110100010001111010011000000100000110000011011000110100011010101100000110110101101111001100100011011111010111011011001110010101010100111000010101011001111011011110011111100010110101111111001010000111111000110001100010111100101010011101011011000001111011110000000100100"

In [18]:
# Inputs: U - information bits + CRC32 + 00 
#         X - 1/2 rate convolutional code
#         A - interleaved convolutional code
#         C - PPM symbols in 16 bit channels
# Output: True
# Exceptions: Thrown when input does not match processed output
def check(U,X,A,C):
    x = outer_enc(U)
    assert x == X, "Outer encode failed"
    a = interleave(x)
    assert a == A, "Interleave failed"
    c = inner_enc(a)
    assert c == C, "Inner encode failed"
    return True

In [7]:
U = to_bool_array(U)
X = to_bool_array(X)
A = to_bool_array(A)
C = to_bool_array(C)

In [31]:
def inner_decode(Y,N):
    alpha = [[None, None] for i in range(N)]
    beta = [[None, None] for i in range(N)]
    alpha[0][0] = 0
    alpha[0][1] = -inf
    beta[nm - 1][0] = beta[nm - 1][1] = log(.5)
    for i in range(1,N):
        

In [36]:
def inner_trellis():
    for i in range(16):
        X = to_bool_array(format(i,'04b'))
        for j in range(2):
            pass

In [12]:
U = "01001000011010010010000100"
U = to_bool_array(U)
X = outer_enc(U)
A = interleave(X)
C = inner_enc(A)
int("0101",2)

5

In [7]:
len(U)
len(C)

15120

In [19]:
hex(0b0101)

'0x5'

In [11]:
for j in range(16):
    A = '{:04b}'.format(j)
    PPM = 0
    for i in range(len(A)):
        PPM = 2*PPM*(int(i%4 != 0)) + (PPM & 1 ^ int(A[i]))
        if i%4 == 3:
            print(A,'{:#01x}'.format(PPM),PPM&1)
            PPM = PPM & 1

0000 0x0 0
0001 0x1 1
0010 0x3 1
0011 0x2 0
0100 0x7 1
0101 0x6 0
0110 0x4 0
0111 0x5 1
1000 0xf 1
1001 0xe 0
1010 0xc 0
1011 0xd 1
1100 0x8 0
1101 0x9 1
1110 0xb 1
1111 0xa 0


In [2]:
inner_enc([0,0,0,1])

[False, False, False, True]